In [1]:
#%%% JSON content script
import pandas as pd
import numpy as np
import json
from sqlalchemy import create_engine

connection_string = 'mssql+pyodbc:///?odbc_connect=' \
                    'Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B' \
                    'Server%3Dmaxreportsrvr.database.windows.net%3B' \
                    'Database%3Dmax_report_db%3B' \
                    'UID%3Dreportadmin%3B' \
                    'PWD%3D#DAff!%nz8r7'\

engine = create_engine(connection_string)

# Create the SQL-query to retrieve the "EmailID," "To," and "Content" columns from the "Emails" table
sql = """SELECT 
            L.MXRITEMNAME,
            L.SALESORDERNUMBER,
            CAST(H.ORDERCREATIONDATETIME AS DATE) as ORDERCREATIONDATE,
            SUM(CASE WHEN L.ORDEREDSALESQUANTITY > 0 THEN L.ORDEREDSALESQUANTITY ELSE 0 END) AS TotalSalesQuantity
        FROM 
            [dbo].[SalesOrderLineV2Staging] L
        JOIN 
            [dbo].[SalesOrderHeaderV2Staging] H ON L.SALESORDERNUMBER = H.SALESORDERNUMBER
        WHERE 
            L.DATAAREAID = 'MNHQ'
            AND (L.MXRITEMNAME = 'Tegel' OR L.MXRITEMNAME = 'Vloerverwarming')
        GROUP BY 
            L.MXRITEMNAME, L.SALESORDERNUMBER, CAST(H.ORDERCREATIONDATETIME AS DATE)
            ORDER BY L.SALESORDERNUMBER;

         """

# Get the email data as a Pandas DataFrame
data = pd.read_sql(sql, engine)

# Close the database connection
engine.dispose()


In [12]:
duplicated_order_numbers = data.pivot_table(index=['SALESORDERNUMBER'], aggfunc='size')[lambda x: x >= 2].index
data = data[data['SALESORDERNUMBER'].isin(duplicated_order_numbers)]

# Initialize the new columns
ex_data = pd.DataFrame()

# Step 2: Fill in the counts
for order_number in duplicated_order_numbers:
    order_data = data[data['SALESORDERNUMBER'] == order_number]
    if order_data['MXRITEMNAME'].iloc[0] == order_data['MXRITEMNAME'].iloc[1]:
        tot_count = order_data['TotalSalesQuantity'].iloc[0] + order_data['TotalSalesQuantity'].iloc[1]
        new_row = {
            'SALESORDERNUMBER': order_number, 
            'ORDERCREATIONDATE': order_data['ORDERCREATIONDATE'].iloc[0], 
            order_data['MXRITEMNAME'].iloc[0]: tot_count, 
            'VloerverwarmingCount': 0
        }
        ex_data = ex_data.append(new_row, ignore_index=True)
    else:
        new_row = {
            'SALESORDERNUMBER': order_number, 
            'ORDERCREATIONDATE': order_data['ORDERCREATIONDATE'].iloc[0], 
            order_data['MXRITEMNAME'].iloc[0]: order_data['TotalSalesQuantity'].iloc[0], 
            order_data['MXRITEMNAME'].iloc[1]: order_data['TotalSalesQuantity'].iloc[1]
        }
        ex_data = ex_data.append(new_row, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [7]:
order_data

,MXRITEMNAME,SALESORDERNUMBER,ORDERCREATIONDATE,TotalSalesQuantity
296,Tegel,MNHQSO0001197,2020-10-05,7.0
297,Tegel,MNHQSO0001197,2020-10-03,7.0


In [ ]:
np.unique(ex_data['Vloerverwarming'])

KeyError: 'Vloerverwarming'

In [ ]:
grouped_data

,SALESORDERNUMBER,TotalSalesQuantity
0,MNHQSO0000002,44.0
1,MNHQSO0000003,62.0
2,MNHQSO0000004,218.0
3,MNHQSO0000006,42.0
4,MNHQSO0000007,140.0
...,...,...
96091,MNHQSO0452619,16.0
96092,MNHQSO0452623,18.0
96093,MNHQSO0452635,46.0
96094,MNHQSO0452638,68.0


In [ ]:
pivoted_data

MXRITEMNAME,SALESORDERNUMBER,ORDERCREATIONDATE,Tegel,Vloerverwarming
0,MNHQSO0000002,2020-09-30,44,0
1,MNHQSO0000003,2020-09-30,62,0
2,MNHQSO0000004,2020-09-30,218,0
3,MNHQSO0000006,2020-09-30,42,0
4,MNHQSO0000007,2020-09-30,140,0
...,...,...,...,...
96154,MNHQSO0452619,2024-01-19,16,0
96155,MNHQSO0452623,2024-01-19,18,0
96156,MNHQSO0452635,2024-01-19,46,0
96157,MNHQSO0452638,2024-01-19,68,0
